## Importing Tensorflow and other utility functions

In [1]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.contrib.data import Dataset

import numpy as np
from tqdm import tqdm

Using TensorFlow backend.


## Dataset Download and Preprocessing

In [2]:
# Loading Dataset
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

# Preprocess and save the data
# Preparing data for CNN model

# Normalization
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255
test_X = test_X / 255

# Converting train_Y and test_Y to 1-hot vectors
train_Y_one_hot = to_categorical(train_y)
test_Y_one_hot = to_categorical(test_y)
print(train_Y_one_hot.shape, test_Y_one_hot.shape)

# Image Shapes - converting to 4-D (for 4-D tensors in CNN) 
train_X_processed = train_X.reshape([-1, 28, 28, 1])
print(train_X_processed.shape)

# Validation Data
train_X_split, valid_X_split, train_Y_split, valid_Y_split = train_test_split(train_X_processed, train_Y_one_hot, test_size=0.2)

# Saving Data
np.save('data/train_X.npy', train_X_split)
np.save('data/train_y.npy', train_Y_split)
np.save('data/valid_X.npy', valid_X_split)
np.save('data/valid_y.npy', valid_Y_split)
np.save('data/test_X.npy', test_X)
np.save('data/test_y.npy', test_Y_one_hot)

(60000, 10) (10000, 10)
(60000, 28, 28, 1)


## Loading data (if data already preprocessed and saved)

In [3]:
# Load Saved Data
train_X, train_y = np.load('data/train_X.npy'), np.load('data/train_y.npy')
test_X, test_y = np.load('data/test_X.npy'), np.load('data/test_y.npy')
valid_X, valid_y = np.load('data/valid_X.npy'), np.load('data/valid_y.npy')
train_X.shape, train_y.shape

((48000, 28, 28, 1), (48000, 10))

## Building Convolutional Neural Network for classification

### Pipeline
1. Define Constants (hyper parameters)
    - Learning rate
    - Batch size
    - Number of epochs
    - Image size
    - Number of classes
2. Define X and y as tf.placeholders
3. Define helper functions for generating weights and biases as tf.Variables
4. Define and build architecture as follows
    - (CNN_8_3x3 + LeakyRelu + MAX Pool_2x2_2)
    - (CNN_16_3x3 + LeakyRelu + MAX Pool_2x2_2)
    - (CNN_32_3x3 + LeakyRelu + MAX Pool_2x2_2)
    - (Flatten)
    - (Dense_128)
    - (Dense_10)
5. Define accuracy operation
6. Define loss operation - softmax + crossentropy
7. Define optimizer - Adam
8. Run Training loop in a session
9. Capture scalar summaries for visualization on TensorBoard
    - Learning rate
    - Loss
    - Accuracy

In [4]:
LOG_DIR = 'tmp/log'

# Constants - Hyper Parameters
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
EPOCHS = 20
IMAGE_SIZE = 28
NUM_CLASS = 10

In [5]:
def weight_variable(shape):
    weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [6]:
# Defining inputs to model
X = tf.placeholder(dtype=tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1])
y = tf.placeholder(dtype=tf.float32)

In [7]:
# Building the model
# Conv 2d + Leaky ReLU + MaxPool
with tf.name_scope(name="CONV_1"):
    W_1 = weight_variable([3, 3, 1, 8])
    b_1 = bias_variable([8])
    conv_layer_1 = tf.nn.conv2d(X, W_1, [1, 1, 1, 1], padding="SAME") + b_1 # (W*X + b)
    leaky_relu_1 = tf.nn.leaky_relu(conv_layer_1, alpha=0.1)
    max_pool_1 = tf.nn.max_pool(leaky_relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME")
    
# Conv 2d + Leaky ReLU + MaxPool
with tf.name_scope(name="CONV_2"):
    W_2 = weight_variable([3, 3, 8, 16])
    b_2 = bias_variable([16])
    conv_layer_2 = tf.nn.conv2d(max_pool_1, W_2, [1, 1, 1, 1], padding="SAME") + b_2 # (W*max_pool1 + b)
    leaky_relu_2 = tf.nn.leaky_relu(conv_layer_2, alpha=0.1)
    max_pool_2 = tf.nn.max_pool(leaky_relu_2, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME")
    
# Conv 2d + Leaky ReLU + MaxPool
with tf.name_scope(name="CONV_3"):
    W_3 = weight_variable([3, 3, 16, 32])
    b_3 = bias_variable([32])
    conv_layer_3 = tf.nn.conv2d(max_pool_2, W_3, [1, 1, 1, 1], padding="SAME") + b_3 # (W*max_pool2 + b)
    leaky_relu_3 = tf.nn.leaky_relu(conv_layer_3, alpha=0.1)
    max_pool_3 = tf.nn.max_pool(leaky_relu_3, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME")

# FLATTEN + Fully connected Layer
with tf.name_scope(name="DENSE"):
    W_fc1 = weight_variable([4 * 4 * 32, 128])
    b_fc1 = bias_variable([128])
    
    pool_flatten = tf.reshape(max_pool_3, [-1, 4 * 4 * 32])
    
    fc_layer_1 = tf.matmul(pool_flatten, W_fc1) + b_fc1
    fc_activated = tf.nn.relu(fc_layer_1)

# Output Layer - Dense
with tf.name_scope(name="Output"):
    W_op = weight_variable([128, NUM_CLASS])
    b_op = bias_variable([NUM_CLASS])
    
    y_ = tf.matmul(fc_activated, W_op) + b_op

In [8]:
# Defining Loss op + optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_op = tf.train.AdamOptimizer().minimize(loss_op)

# Evaluations
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Init op
init_op = tf.global_variables_initializer()

# Summary Merge op
merged = tf.summary.merge_all()

In [9]:
# Training loop
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    sess.run(init_op)
    for epoch in range(EPOCHS):
        print('Epoch {}'.format(epoch + 1))
        
        for index, offset in tqdm(list(enumerate(range(0, train_X.shape[0], BATCH_SIZE))), ncols=100):
            xs, ys = train_X[offset: offset + BATCH_SIZE], train_y[offset: offset + BATCH_SIZE]
            sess.run(train_op, feed_dict={
                X: xs,
                y: ys
            })
        
        train_accuracy = accuracy.eval(feed_dict={
            X: train_X,
            y: train_y
        })
        validation_accuracy = accuracy.eval(feed_dict={
            X: valid_X,
            y: valid_y
        })
        print('Training Accuracy: {}%\nValidation Accuracy: {}%\n'.format(train_accuracy, validation_accuracy))
    summary_writer.close()

Epoch 1


100%|█████████████████████████████████████████████████████████████| 750/750 [00:45<00:00, 16.63it/s]


Training Accuracy: 0.8488749861717224%
Validation Accuracy: 0.8460000157356262%

Epoch 2


100%|█████████████████████████████████████████████████████████████| 750/750 [00:45<00:00, 16.49it/s]


Training Accuracy: 0.8747291564941406%
Validation Accuracy: 0.8696666955947876%

Epoch 3


...

Training Accuracy: 0.9388958215713501%
Validation Accuracy: 0.9079999923706055%

Epoch 19


100%|█████████████████████████████████████████████████████████████| 750/750 [00:43<00:00, 17.22it/s]


Training Accuracy: 0.941979169845581%
Validation Accuracy: 0.9080833196640015%

Epoch 20


100%|█████████████████████████████████████████████████████████████| 750/750 [00:41<00:00, 18.06it/s]


Training Accuracy: 0.942270815372467%
Validation Accuracy: 0.9051666855812073%

